In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets
import cv2

In [2]:
# Función para cargar la imagen
def load_image(image_buffer):
    # Convertir el buffer de bytes en una imagen usando OpenCV
    image = np.frombuffer(image_buffer, np.uint8)
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

In [3]:
# Función para calcular la distancia euclidiana
def euclidean_distance(a, b):
    return np.sqrt(np.sum((a - b) ** 2, axis=1))

In [4]:
# Implementación desde cero de KMeans
def kmeans_custom(image, k, num_iterations=10):
    # Redimensionar la imagen a una lista de píxeles
    pixel_values = image.reshape((-1, 3))
    pixel_values = np.float32(pixel_values)

    # Inicializar los centroides de forma aleatoria
    centroids = pixel_values[np.random.choice(pixel_values.shape[0], k, replace=False)]

    frames = []  # Lista para almacenar frames de cada iteración
    
    for i in range(num_iterations):
        # Etapa de asignación: asignar cada píxel al centroide más cercano
        distances = np.array([euclidean_distance(pixel_values, centroid) for centroid in centroids])
        labels = np.argmin(distances, axis=0)

        # Guardar frame de la imagen segmentada en esta iteración
        segmented_image = centroids[labels].reshape(image.shape)
        
        # Asegurarse de que los valores están en el rango 0-255
        segmented_image = np.clip(segmented_image, 0, 255).astype(np.uint8)
        frames.append(segmented_image)

        # Etapa de actualización: recalcular los centroides
        new_centroids = np.array([pixel_values[labels == j].mean(axis=0) if np.any(labels == j) else centroids[j] for j in range(k)])

        # Verificar convergencia (si los centroides no cambian)
        if np.all(centroids == new_centroids):
            break
        centroids = new_centroids

    return centroids, labels, frames

In [5]:
# Función para visualizar la evolución de KMeans
def plot_kmeans_evolution(image, k):
    centroids, labels, frames = kmeans_custom(image, k)
    
    # Mostrar los frames de la evolución
    for i, frame in enumerate(frames):
        plt.figure(figsize=(6, 6))
        plt.imshow(frame)
        plt.title(f'Frame {i + 1}: Iteración {i + 1}')
        plt.axis('off')
        plt.show()

In [6]:
# Función para cargar la imagen desde el input interactivo
def upload_and_process_image(image_upload, k):
    if image_upload:
        # Acceder al archivo cargado directamente desde el widget
        uploaded_file = image_upload[0]
        image_buffer = uploaded_file['content']
        image = load_image(image_buffer)
        
        plot_kmeans_evolution(image, k)

In [11]:
# Crear un widget interactivo
def interactive_kmeans():
    # Widget para cargar una imagen
    image_upload = widgets.FileUpload(accept='.jpg, .png', multiple=False)

    # Slider para seleccionar el valor de k
    k_slider = widgets.IntSlider(value=2, min=2, max=100, step=1, description='k clusters')

    # Usar interact para la funcionalidad interactiva
    interact(upload_and_process_image, image_upload=image_upload, k=k_slider)

In [12]:
# Ejecutar el widget interactivo
interactive_kmeans()

interactive(children=(FileUpload(value=(), accept='.jpg, .png', description='Upload'), IntSlider(value=2, desc…